In [29]:
from ews.models import *
import pandas as pd
import geojson
import numpy as np
from django_pandas.io import read_frame

In [8]:
d = pd.read_csv('D:/DWC_plattform/riverflow_locations.csv')

In [9]:
d

,SiteName,SiteReferencementName,SiteType,SourceID,SiteLatitude(y),SiteLongitude(x)
0,Alfortville,alfortville_seine,Riverflow,NaN,48.797558,2.418510
1,Austerllitz,austerlitz,Riverflow,NaN,48.845091,2.366067
2,Creteil,cr�teil_marne,Riverflow,NaN,48.790437,2.472380
3,Gournay-sur-Marne,gournay_sur_marne_marne,Riverflow,NaN,48.865347,2.574696
4,Saint-Fargeau-Ponthierry,saint_fargeau_ponthierry_marne,Riverflow,NaN,48.793223,2.739247


In [10]:
for index, row in d.iterrows():
    loc = Site()
    loc.name = row['SiteName']
    loc.ref_name = row['SiteReferencementName']
    loc.geom = {"coordinates": [row['SiteLongitude(x)'],row['SiteLatitude(y)']], "type": "Point"}
    loc.feature_type = FeatureType.objects.get(name = row['SiteType'])
    loc.owner = User.objects.get(id = 1)
    try:
        loc.save()
    except:
        print(loc)

In [8]:
rf = pd.read_csv('D:/DigitalWaterCity/data_processed/river_flows.csv', parse_dates = True, index_col = "Date")

In [9]:
rf = rf[rf["name"]=="Austerlitz"]

In [10]:
rf

,Q (m3/s),name,river
Date,,,
2014-01-01 00:00:00+00:00,615.0,Austerlitz,Seine
2014-01-01 01:00:00+00:00,618.0,Austerlitz,Seine
2014-01-01 02:00:00+00:00,616.0,Austerlitz,Seine
2014-01-01 03:00:00+00:00,613.0,Austerlitz,Seine
2014-01-01 04:00:00+00:00,610.0,Austerlitz,Seine
...,...,...,...
2020-09-14 06:00:00+00:00,103.0,Austerlitz,Seine
2020-09-14 07:00:00+00:00,105.0,Austerlitz,Seine
2020-09-14 08:00:00+00:00,111.0,Austerlitz,Seine


In [11]:
rf = rf.groupby("name").resample("D").mean().reset_index()

In [32]:
rf["Q (m3/s)"] = rf["Q (m3/s)"].round(2)

In [37]:
rf = rf.dropna()

In [38]:
rf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2440 entries, 0 to 2448
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype              
---  ------    --------------  -----              
 0   name      2440 non-null   object             
 1   Date      2440 non-null   datetime64[ns, UTC]
 2   Q (m3/s)  2440 non-null   float64            
dtypes: datetime64[ns, UTC](1), float64(1), object(1)
memory usage: 76.2+ KB


In [39]:
for index, row in rf.iterrows():
    dpoint = FeatureData()
    dpoint.date = row["Date"]
    dpoint.value = row["Q (m3/s)"]
    dpoint.site = Site.objects.get(name = row['name'])
    dpoint.save()
    

In [40]:
FeatureData.objects.filter(site = Site.objects.get(name = "Austerlitz"))

<QuerySet [<FeatureData: FeatureData object (265484)>, <FeatureData: FeatureData object (265485)>, <FeatureData: FeatureData object (265486)>, <FeatureData: FeatureData object (265487)>, <FeatureData: FeatureData object (265488)>, <FeatureData: FeatureData object (265489)>, <FeatureData: FeatureData object (265490)>, <FeatureData: FeatureData object (265491)>, <FeatureData: FeatureData object (265492)>, <FeatureData: FeatureData object (265493)>, <FeatureData: FeatureData object (265494)>, <FeatureData: FeatureData object (265495)>, <FeatureData: FeatureData object (265496)>, <FeatureData: FeatureData object (265497)>, <FeatureData: FeatureData object (265498)>, <FeatureData: FeatureData object (265499)>, <FeatureData: FeatureData object (265500)>, <FeatureData: FeatureData object (265501)>, <FeatureData: FeatureData object (265502)>, <FeatureData: FeatureData object (265503)>, '...(remaining elements truncated)...']>

In [41]:
read_frame(FeatureData.objects.filter(site = Site.objects.get(name = "Austerlitz")))

,id,date,value,site
0,265484,2014-01-01 00:00:00+00:00,594.580,Austerlitz
1,265485,2014-01-02 00:00:00+00:00,558.460,Austerlitz
2,265486,2014-01-03 00:00:00+00:00,576.500,Austerlitz
3,265487,2014-01-04 00:00:00+00:00,618.830,Austerlitz
4,265488,2014-01-05 00:00:00+00:00,598.830,Austerlitz
...,...,...,...,...
2435,267919,2020-09-10 00:00:00+00:00,106.830,Austerlitz
2436,267920,2020-09-11 00:00:00+00:00,108.960,Austerlitz
2437,267921,2020-09-12 00:00:00+00:00,104.460,Austerlitz
2438,267922,2020-09-13 00:00:00+00:00,106.880,Austerlitz
